# Export simulation data over sensors by seasons
- This script is used to hourly mean simulation data by seasons.
- Simulations: CNTL_PT, LCZ_PT.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
home_path = '/gws/nopw/j04/duicv/yuansun/'
output_dir = home_path + '0_lcz_mcr/output_analysis/single_point/variation_diurnal_season/'
df_meta = pd.read_csv(home_path + '0_lcz_mcr/output_analysis/single_point/spinup_analysis_period/sensor_metadata.csv')
data_dir = home_path + '0_lcz_mcr/output_analysis/single_point/simulation/sensor_output/'
season_list = ['DJF', 'MAM', 'JJA', 'SON']

In [3]:
for j, season in enumerate(season_list):
    hour_average = []
    for i, sn in enumerate(df_meta['sensor']):
        df = pd.read_csv(data_dir + sn + '.csv')  
        df['timestamp'] = pd.to_datetime(df['timestamp']) 
        df.set_index('timestamp', inplace=True)
        df_season = df[df.index.month.isin([12, 1, 2])] if season == 'DJF' else \
                    df[df.index.month.isin([3, 4, 5])] if season == 'MAM' else \
                    df[df.index.month.isin([6, 7, 8])] if season == 'JJA' else \
                    df[df.index.month.isin([9, 10, 11])]
        hourly_avg = df_season.groupby(df_season.index.hour).mean()
        hourly_avg['sensor'] = sn
        hour_average.append(hourly_avg)
    hour_average_df = pd.concat(hour_average, axis=0).reset_index()
    hour_average_df.to_csv(output_dir + season + '_hourly_mean.csv', index=False)